In [118]:
library(tidyverse)
library(lubridate)
library(forecast) # Para modelos AR

In [642]:
df<-read_csv('df.csv',col_names = TRUE,cols(
  Ano = col_double(),
  Mes = col_double(),
  Ano_Mes = col_character(),
  Fecha = col_date(format = ""),
  Precios_Mensuales = col_double(),
  Aportes_Energia_GWh = col_double(),
  Volumen_Util_Diario_Energia_GWh = col_double(),
  Demanda_Energia_SIN_GWh = col_double(),
  ONI = col_double(),
  CEN_Total = col_double(),
  CEN_Hidro = col_double(),
  CEN_Termica = col_double()
))

# Creacion de funciones

In [422]:
oni<-function(df, start_at){
    
    fechaini<-as.Date(start_at)
    
    n_pronosticos<-df%>%filter(Fecha>=start_at)%>%select(ONI)%>%nrow
    
    
    while(any(is.na(df$ONI))){
        fila<-sum(complete.cases(df$ONI))+1
        m<-auto.arima(unlist(unname(df[1:fila-1,'ONI'])),max.q = 0,max.Q = 0,max.d = 0,max.D = 0)
        y_est<-round(as.vector(unlist(forecast(m, 24)['mean'])),1)
        df$ONI[is.na(df$ONI)]<-y_est
    }
    #df$ONI[df$Fecha>=as.Date(start_at)]<-round(runif(n_pronosticos,-0.5,0.5),1)
    
    return(df)
}
#ok

In [634]:
aportes<-function(df, start_at, rezagos_oni=12,rezagos_aportes=24){
    
    fechaini<-as.Date(start_at)
    #n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(Aportes_Energia_GWh)%>%nrow
    #df$Aportes_Energia_GWh[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,min(df$Aportes_Energia_GWh,na.rm = TRUE),max(df$Aportes_Energia_GWh,na.rm = TRUE)),3)

    fecha_forecast<-seq.Date(from = as.Date(fechaini),length.out = 24,by = 'month') #Parametro!!!

    for(i in seq(length(fecha_forecast))){
    
        temp<-df%>%select(Fecha, Aportes_Energia_GWh, ONI, Aportes_Normalizados)
        rezagos_oni<-rezagos_oni                                                          #Rezagos para el ONI (parametrizable)
        rezagos_aportes<-rezagos_aportes                                                      #Rezagos para los Aportes (parametrizable)

        for(j in seq(rezagos_oni)){                                             #Creacion de columnas rezagadas ONI
            col_oni<-paste0('ONI_',j)                                           #   
            temp[[col_oni]]<-lag(df$ONI,j)                                      #
        }                                                                       #

        for(k in seq(rezagos_aportes)){                                         #Creacion de columnas rezagadas Aportes
            col_aportes<-paste0('Aportes_',k)                                   #
            temp[[col_aportes]]<-lag(df$Aportes_Normalizados,k)                 #
        }                                                                       #


        #new_data<-temp%>%filter(Fecha>=as.Date('2019-06-01')%m+%months(-24)
        new_data<-temp
        
        
        
        temp<-temp%>%select(-1,-2,-3,Aportes_Normalizados,                      #Elimina Fecha, Aportes_Energia, ONI
                                    starts_with('Aportes_[0-9]'),               #Trae los rezagos de Aportes
                                    starts_with('ONI_'))            #Trae los rezagos de ONI


        #new_data<-temp

        temp<-temp%>%drop_na()

        #Aplicamos un modelo Forward Selection
        m_FitAll<-lm(Aportes_Normalizados~.,data = temp)
        m<-(step(lm(Aportes_Normalizados~1,data=temp),direction = 'forward',scope = formula(m_FitAll),trace = 0))

        # Seleccionar solo variables significativas
        toselect.x<-summary(m)$coeff[,4] < 0.01
        relevant.x <- names(toselect.x)[toselect.x == TRUE] 
        sig.formula <- as.formula(paste("Aportes_Normalizados ~",paste(relevant.x,collapse = '+'))) 

        m<-lm(formula = sig.formula,data = temp)
        
        y_est<-round(as.vector(unlist(unname(forecast(m,h = 1,newdata = new_data%>%filter(Fecha==fecha_forecast[i]%m+%months(-1))%>%select(Fecha, one_of(names(summary(m)$coeff[-1,1])))%>%select(-Fecha))['mean']))),4)
        
        #print(paste('para el mes', fecha_forecast[i],'se estimo un valor de', y_est,collapse = '/n' ))
        df$Aportes_Normalizados[df$Fecha==fecha_forecast[i]]<-y_est

        

        #new_data<-new_data%>%select(Fecha, one_of(names(summary(m)$coeff[-1,1])))%>%filter(Fecha>='2019-06-01')%>%select(-Fecha)


#        df$Aportes_Normalizados[is.na(df$Aportes_Normalizados)]<-round(as.vector(unlist(unname(forecast(m,h = 24,newdata = new_data)['mean']))),2)

#         while(df$Aportes_Normalizados[!complete.cases(df$Aportes_Normalizados)]%>%length>0){
#             #df[!complete.cases(df),'Aportes_Normalizados'][[1]][1]
#             df$Aportes_Normalizados[is.na(df$Aportes_Normalizados)][1]<-unlist(unname(forecast(object = m, h = 1,newdata = new_data%>%tail(25)%>%head(1)%>%select(-1,everything()))['mean']))

#             new_data<-new_data%>%drop_na()
    }


    
    return(df)
}

In [424]:
reservas<-function(df, start_at){
    
    fechaini<-as.Date(start_at)
    n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(Volumen_Util_Diario_Energia_GWh)%>%nrow
    df$Volumen_Util_Diario_Energia_GWh[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,min(df$Volumen_Util_Diario_Energia_GWh,na.rm = TRUE),max(df$Volumen_Util_Diario_Energia_GWh,na.rm = TRUE)),3)
    
    return(df)
}

In [425]:
precios<-function(df, start_at){
    
    fechaini<-as.Date(start_at)
    n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(Precios_Mensuales)%>%nrow
    df$Precios_Mensuales[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,min(df$Precios_Mensuales,na.rm = TRUE),max(df$Precios_Mensuales,na.rm = TRUE)),3)
    
    return(df)
}

In [426]:
demanda<-function(df, start_at){
    
    fechaini<-as.Date(start_at)
    n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(Demanda_Energia_SIN_GWh)%>%nrow
    df$Demanda_Energia_SIN_GWh[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,min(df$Demanda_Energia_SIN_GWh,na.rm = TRUE),max(df$Demanda_Energia_SIN_GWh,na.rm = TRUE)),3)
    
    return(df)
}

In [427]:
columnas_calculadas<-function(df){
    #debo montar dem_CEN
    df$Dem_CEN<-round(df$Demanda_Energia_SIN_GWh/(as.vector(days_in_month(df$Fecha))*df$CEN_Total*24),4)
    #debo montar Term_Hidro
    df$Termica_Hidraulica<-round(df$CEN_Termica/df$CEN_Hidro,4)
    
    return(df)
}

In [428]:
normalizar_hidrologia<-function(df){
    df<-df%>%
        left_join(
                df%>%arrange(Mes)%>%group_by(Mes)%>%summarize(mean_aportes = mean(Aportes_Energia_GWh,na.rm = TRUE), 
                                                              mean_reservas = mean(Volumen_Util_Diario_Energia_GWh,na.rm = TRUE),
                                                              sd_aportes = sd(Aportes_Energia_GWh, na.rm = TRUE),
                                                              sd_reservas = sd(Volumen_Util_Diario_Energia_GWh,na.rm = TRUE)),by='Mes')%>%
    mutate(
        Aportes_Normalizados = (Aportes_Energia_GWh-mean_aportes)/sd_aportes,
        Reservas_Normalizadas = (Volumen_Util_Diario_Energia_GWh-mean_reservas)/sd_reservas)
    
    return(df)
}

# Probar funciones

In [637]:
df<-normalizar_hidrologia(df)
df<-oni(df,start_at = '2019-06-01')
df<-aportes(df,start_at = '2019-06-01',rezagos_oni = 12,rezagos_aportes = 24)
# df<-reservas(df,start_at = '2019-06-01')
#df<-demanda(df,start_at = '2019-06-01')
#df<-columnas_calculadas(df)
#df<-precios(df,start_at = '2019-06-01')

In [639]:
# #Definir el dataframe Aportes, ONI

# fecha_forecast<-seq.Date(from = as.Date('2019-06-01'),length.out = 24,by = 'month') #Parametro!!!

# for(i in seq(length(fecha_forecast))){
    
#     temp<-df%>%select(Fecha, Aportes_Energia_GWh, ONI, Aportes_Normalizados)
#     rezagos_oni<-12                                                          #Rezagos para el ONI (parametrizable)
#     rezagos_aportes<-24                                                      #Rezagos para los Aportes (parametrizable)

#         for(j in seq(rezagos_oni)){                                             #Creacion de columnas rezagadas ONI
#             col_oni<-paste0('ONI_',j)                                           #   
#             temp[[col_oni]]<-lag(df$ONI,j)                                      #
#         }                                                                       #

#         for(k in seq(rezagos_aportes)){                                         #Creacion de columnas rezagadas Aportes
#             col_aportes<-paste0('Aportes_',k)                                   #
#             temp[[col_aportes]]<-lag(df$Aportes_Normalizados,k)                 #
#         }                                                                       #


#         #new_data<-temp%>%filter(Fecha>=as.Date('2019-06-01')%m+%months(-24)
#         new_data<-temp
        
        
        
#         temp<-temp%>%select(-1,-2,-3,Aportes_Normalizados,                      #Elimina Fecha, Aportes_Energia, ONI
#                                     starts_with('Aportes_[0-9]'),               #Trae los rezagos de Aportes
#                                     starts_with('ONI_'))            #Trae los rezagos de ONI


#         #new_data<-temp

#         temp<-temp%>%drop_na()

#         #Aplicamos un modelo Forward Selection
#         m_FitAll<-lm(Aportes_Normalizados~.,data = temp)
#         m<-(step(lm(Aportes_Normalizados~1,data=temp),direction = 'forward',scope = formula(m_FitAll),trace = 0))

#         # Seleccionar solo variables significativas
#         toselect.x<-summary(m)$coeff[,4] < 0.01
#         relevant.x <- names(toselect.x)[toselect.x == TRUE] 
#         sig.formula <- as.formula(paste("Aportes_Normalizados ~",paste(relevant.x,collapse = '+'))) 

#         m<-lm(formula = sig.formula,data = temp)
        
#         y_est<-round(as.vector(unlist(unname(forecast(m,h = 1,newdata = new_data%>%filter(Fecha==fecha_forecast[i]%m+%months(-1))%>%select(Fecha, one_of(names(summary(m)$coeff[-1,1])))%>%select(-Fecha))['mean']))),4)
        
#         print(paste('para el mes', fecha_forecast[i],'se estimo un valor de', y_est,collapse = '/n' ))
#         df$Aportes_Normalizados[df$Fecha==fecha_forecast[i]]<-y_est

        

#         #new_data<-new_data%>%select(Fecha, one_of(names(summary(m)$coeff[-1,1])))%>%filter(Fecha>='2019-06-01')%>%select(-Fecha)


# #        df$Aportes_Normalizados[is.na(df$Aportes_Normalizados)]<-round(as.vector(unlist(unname(forecast(m,h = 24,newdata = new_data)['mean']))),2)

# #         while(df$Aportes_Normalizados[!complete.cases(df$Aportes_Normalizados)]%>%length>0){
# #             #df[!complete.cases(df),'Aportes_Normalizados'][[1]][1]
# #             df$Aportes_Normalizados[is.na(df$Aportes_Normalizados)][1]<-unlist(unname(forecast(object = m, h = 1,newdata = new_data%>%tail(25)%>%head(1)%>%select(-1,everything()))['mean']))

# #             new_data<-new_data%>%drop_na()
#         }
# #return(df)

In [670]:
generar_archivos<-function(df,
                           start_at,
                           n_iter,
                           folder_location){
    
    
    if(dir.exists(folder_location)){
        while(file.remove(list.files(path = folder_location,pattern = '[0-9][0-9][0-9][0-9][0-9].csv',full.names = TRUE)[1])==TRUE){
            list.files(path = folder_location,pattern = '[0-9][0-9][0-9][0-9][0-9].csv',full.names = TRUE)[1]
        }

        fechaini<-as.Date(start_at)

        for( i in seq(n_iter)){
            ini<-now()
#             #Normalizar aportes
#             #normalizar reservas
#             #Agregar variables calculadas
#             df<-precios(df,fechaini)
#             df<-oni(df,fechaini)
#             df<-aportes(df,fechaini)
#             df<-reservas(df,fechaini)
#             df<-demanda(df,fechaini)
#             df<-columnas_calculadas(df)
            df<-read_csv('df.csv',col_names = TRUE,cols(
                          Ano = col_double(),
                          Mes = col_double(),
                          Ano_Mes = col_character(),
                          Fecha = col_date(format = ""),
                          Precios_Mensuales = col_double(),
                          Aportes_Energia_GWh = col_double(),
                          Volumen_Util_Diario_Energia_GWh = col_double(),
                          Demanda_Energia_SIN_GWh = col_double(),
                          ONI = col_double(),
                          CEN_Total = col_double(),
                          CEN_Hidro = col_double(),
                          CEN_Termica = col_double()
                        ))
            
            df<-normalizar_hidrologia(df)
            df<-oni(df,start_at = '2019-06-01')
            df<-aportes(df,start_at = '2019-06-01',rezagos_oni = 12,rezagos_aportes = 24)
            df<-reservas(df,start_at = '2019-06-01')
            df<-demanda(df,start_at = '2019-06-01')
            df<-columnas_calculadas(df)
            df<-precios(df,start_at = '2019-06-01')

            #pronostico de variables calculadas
            write_csv(df, paste0(folder_location,str_pad(i, pad = 0, width = 5),'.csv'))
            fin<-now()
            cat(paste0(round(fin-ini,4), ' segundos ',str_pad(i, pad = 0, width = 5),'.csv'),collapse='\n')
        }
        cat(paste0(n_iter, ' archivos generados en la ruta ',folder_location))
    } else{
        cat('Error: Directorio no existe en esta máquina')
    }

}

In [671]:
generar_archivos(df,start_at = '2019-06-01',n_iter = 2,folder_location = './Resultados/')

3.6822 segundos 00001.csv 
3.6774 segundos 00002.csv 
2 archivos generados en la ruta ./Resultados/

# Tareas

In [235]:
#hacer el arx pronosticar uno a uno y tiene como entrada el ONI y se devuelven las unidades 2h
#hacer arx para reservas con oni, aportes, relacion T/H y me devuelvo 2h
#un modelo simple de precio 2h